# Testing custom CNN trained on padded and translated MNIST train set on affNIST test set

In [1]:
from keras import backend as K
from keras.datasets import mnist
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Conv2D, Activation, Dense, Dropout, Lambda
from keras.layers import BatchNormalization, MaxPooling2D, Flatten, Conv1D
from convcaps.capslayers import ConvertToCaps, Conv2DCaps, FlattenCaps
from convcaps.capslayers import DenseCaps, CapsToScalars
from keras.utils import to_categorical
from keras.callbacks import Callback, ModelCheckpoint, TensorBoard
from keras import optimizers
from keras import regularizers
from keras import losses
import numpy as np
from scipy.io import matlab
import os

Using TensorFlow backend.


## Load train set generated in generate_datasets.py

In [2]:
img_rows, img_cols = 40, 40
num_classes = 10

x_train = np.load('generateddatasets/x_train_only_translation.npy').astype(np.float32)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_train /= 255.0

y_train = np.load('generateddatasets/y_train_only_translation.npy')
y_train = to_categorical(y_train, num_classes)

input_shape = (img_rows, img_cols, 1)

## Assemble CNN with 3 conv layers and two dense layers

In [3]:
l2 = regularizers.l2(l=0.001)

inp = Input(shape=input_shape)
l = inp

l = Conv2D(8, (3, 3), activation='relu', kernel_regularizer=l2)(l)
l = BatchNormalization()(l)
l = MaxPooling2D((2, 2))(l)
l = Conv2D(16, (3, 3), activation='relu', kernel_regularizer=l2)(l) 
l = BatchNormalization()(l)
l = MaxPooling2D((2, 2))(l)
l = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2)(l)
l = BatchNormalization()(l)
l = MaxPooling2D((2, 2))(l)
l = Flatten()(l)
l = Dense(72, activation='relu', kernel_regularizer=l2)(l)
l = BatchNormalization()(l)
l = Dense(10, activation='softmax', kernel_regularizer=l2)(l)

model = Model(inputs=inp, outputs=l, name='40x40_input_cnn')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40, 40, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 38, 38, 8)         80        
_________________________________________________________________
batch_normalization_1 (Batch (None, 38, 38, 8)         32        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 19, 19, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 16)        1168      
_________________________________________________________________
batch_normalization_2 (Batch (None, 17, 17, 16)        64        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 16)          0         
__________

In [4]:
def caps_objective(y_true, y_pred):
    return K.sum(y_true * K.clip(0.8 - y_pred, 0, 1) ** 2 + 0.5 * (1 - y_true) * K.clip(y_pred - 0.3, 0, 1) ** 2)

optimizer = optimizers.Adam(lr=0.001)
model.compile(loss=losses.categorical_crossentropy, #caps_objective,
              optimizer=optimizer,
              metrics=['accuracy'])

In [5]:
w_path = os.path.join('weights', model.name)

if not os.path.exists(w_path):
    os.makedirs(w_path)
    
f_name = os.path.join(w_path, 'weights.{epoch:02d}.hdf5')    

# Train for 1 epochs

In [6]:
batch_size = 32
num_epochs = 4

h = model.fit(x_train, y_train,
          batch_size=batch_size, epochs=num_epochs, initial_epoch=0,
          verbose=1, validation_split=0.09,
          callbacks=[ModelCheckpoint(f_name)])

Train on 436800 samples, validate on 43200 samples
Epoch 1/4
436800/436800 [==============================] - 578s - loss: 0.2089 - acc: 0.9642 - val_loss: 0.1318 - val_acc: 0.9805
Epoch 2/4
436800/436800 [==============================] - 538s - loss: 0.1285 - acc: 0.9800 - val_loss: 0.1322 - val_acc: 0.9795
Epoch 3/4
436800/436800 [==============================] - 528s - loss: 0.1193 - acc: 0.9813 - val_loss: 0.1012 - val_acc: 0.9864
Epoch 4/4
436800/436800 [==============================] - 537s - loss: 0.1129 - acc: 0.9820 - val_loss: 0.1042 - val_acc: 0.9851


## Test on affNIST test set

In [16]:
f = matlab.loadmat('affnist/test.mat')

x_test = f['affNISTdata'][0, 0][2].transpose().astype(np.float32)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_test /= 255
input_shape = (img_rows, img_cols, 1)

y_test = f['affNISTdata'][0, 0][4].transpose()

score = model.evaluate(x_test, y_test, verbose=1)
print('')
print('Test score: ', score[0])
print('Test accuracy: ', score[1])    

320000/320000 [==============================] - 98s    

Test score:  0.965831407426
Test accuracy:  0.73925
